In [4]:
import time
from langchain_community.chat_models import ChatOllama
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
import ChatTestDB as log
import Harry_Potter_questions as HP
import importlib

<h2>Testing af forskellige modeller</h2>
<p>Vi har testet Mistral, "all-mpnet-base-v2" og LLAMA3.</p>
Vi valgte at gå med LLAMA3, baseret på response time

In [26]:
#llm = ChatOllama(model="tomasonjo/llama3-text2cypher-demo")

In [4]:
llm = ChatOllama(model="llama3")

In [19]:
#llm = ChatOllama(model="mistral")

In [34]:
#llm = Ollama(model="mistral", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [39]:
#llm = Ollama(model="llama3", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [5]:
# Neo4J Connection 
url = "bolt://localhost:7687"
username = "neo4j"
password = "pleaseletmein"

In [6]:
# Valg af model til test
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

In [7]:
# Tilknyttet model til at lave embeddings
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [8]:
# Oprettet embeddings/vector ud fra Neo4J database data
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url=url,
    username=username,
    password=password,
    index_name='persons',
    node_label=["Person", 'Location', 'Skill', 'Organization', 'Award', "Country", 'Religion' ],
    text_node_properties=['name', "positionHeld", "causeOfDeath", "dateOfBirth", "numberOfChildren", "academicDegree", "dateOfDeath", "age", "productType", "foundingDate" ],
    embedding_node_property='embedding',
)

In [29]:
# Test af svar
response = vector_index.similarity_search(
    "who is Merope Gaunts son?"
)
print(response)

[Document(page_content='\nname: Merope Gaunt\npositionHeld: \ncauseOfDeath: \ndateOfBirth: \nnumberOfChildren: \nacademicDegree: \ndateOfDeath: \nage: \nproductType: \nfoundingDate: '), Document(page_content='\nname: Morfin Gaunt\npositionHeld: \ncauseOfDeath: \ndateOfBirth: \nnumberOfChildren: \nacademicDegree: \ndateOfDeath: \nage: \nproductType: \nfoundingDate: '), Document(page_content='\nname: Malfoy\npositionHeld: \ncauseOfDeath: \ndateOfBirth: \nnumberOfChildren: \nacademicDegree: \ndateOfDeath: \nage: \nproductType: \nfoundingDate: '), Document(page_content='\nname: Henry VIII\npositionHeld: \ncauseOfDeath: \ndateOfBirth: \nnumberOfChildren: \nacademicDegree: \ndateOfDeath: \nage: \nproductType: \nfoundingDate: ')]


# LLM

In [9]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 

{context}

Question: {question}

Helpful Answer:
"""

In [10]:
# Oprettelse af prompt template 
prompt = PromptTemplate.from_template(template)

In [40]:
# Opsættelse af chatbot med LLM og Graph data, fra Neo4J databasen
vector_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_index.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [41]:
# Test af spørgsmål 
start = time.time()
res = vector_qa.invoke("who is Merope Gaunts son?")
end = time.time()
print(res['result'])
print((end - start)/60)

I don't know the answer to that question because there is no information provided about Merope Gaunt's children. Thanks for asking!I don't know the answer to that question because there is no information provided about Merope Gaunt's children. Thanks for asking!
8.99040089448293


In [42]:
rag_chain = (
    {"context": vector_index.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [43]:
start = time.time()
res = rag_chain.invoke("who is Merope Gaunts son?")
end = time.time()
print(res)
print((end - start)/60)

I don't have enough information to answer that question. The provided documents only contain general information about individuals, but not specific relationships or family connections. Therefore, I cannot determine the parentage of Merope Gaunt's children. Thanks for asking!I don't have enough information to answer that question. The provided documents only contain general information about individuals, but not specific relationships or family connections. Therefore, I cannot determine the parentage of Merope Gaunt's children. Thanks for asking!
10.052893205483754


<h3>Test af 20 forskellige spørgsmål til chatbot</h3>

In [11]:
vector_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_index.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [15]:
importlib.reload(log)

<module 'ChatTestDB' from 'C:\\Users\\linew\\Documents\\SOFT_sem2\\Machine_Learning\\ai_exam\\ChatTestDB.py'>

In [65]:
log.LogChat(3, "graph", HP.question1, "test", 1)

In [44]:
start = time.time()
res = vector_qa.invoke(HP.question1)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question1, res['result'], times)
print(res['result'])
print(times)

According to the provided context, there is no information about Harry Potter's wand. Therefore, I don't know.

Thanks for asking!
1.3826561053593953


In [45]:
start = time.time()
res = vector_qa.invoke(HP.question2)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question2, res['result'], times)
print(res['result'])
print(times)

Based on the provided context, there is no information about Riddle-Harry. However, I can tell you that Harry Potter's Hogwarts house is Gryffindor.

Thanks for asking!
1.4154738465944925


In [46]:
start = time.time()
res = vector_qa.invoke(HP.question3)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question3, res['result'], times)
print(res['result'])
print(times)

Merope Gaunt was Bellatrix's mother. Thanks for asking!
1.2134500026702881


In [47]:
start = time.time()
res = vector_qa.invoke(HP.question4)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question4, res['result'], times)
print(res['result'])
print(times)

Merope Gaunt's son is Morfin Gaunt. Thanks for asking!
1.320678448677063


In [48]:
start = time.time()
res = vector_qa.invoke(HP.question5)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question5, res['result'], times)
print(res['result'])
print(times)

Based on the given context, it appears that Scorpius Malfoy is a character related to Severus Snape. James Potter and Remus Lupin are known for creating the Marauder's Map.

Answer: James Potter and Remus Lupin created the Marauder's Map.

Thanks for asking!
1.9377043485641479


In [49]:
start = time.time()
res = vector_qa.invoke(HP.question6)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question6, res['result'], times)
print(res['result'])
print(times)

The Half-Blood Prince! That would be Severus Snape, a Potions Master. Thanks for asking!
1.2251324295997619


In [50]:
start = time.time()
res = vector_qa.invoke(HP.question7)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question7, res['result'], times)
print(res['result'])
print(times)

According to the Harry Potter series, Draco Malfoy and his descendants are considered the heirs of Salazar Slytherin. Thanks for asking!
1.3162723739941915


In [51]:
start = time.time()
res = vector_qa.invoke(HP.question8)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question8, res['result'], times)
print(res['result'])
print(times)

According to the provided context, Albus Dumbledore was the Headmaster of Hogwarts. Thanks for asking!
1.3256601333618163


In [52]:
start = time.time()
res = vector_qa.invoke(HP.question9)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question9, res['result'], times)
print(res['result'])
print(times)

Voldemort created 7 Horcruxes. Thanks for asking!
1.3104245742162068


In [53]:
start = time.time()
res = vector_qa.invoke(HP.question10)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question10, res['result'], times)
print(res['result'])
print(times)

There is no information provided about Ravenclaw, Draco Malfoy, or Slytherin in terms of their positions held, product type, or founding date. However, according to the Harry Potter series by J.K. Rowling, there are three Deathly Hallows: the Elder Wand, the Resurrection Stone, and the Cloak of Invisibility.

Thanks for asking!
2.1068972826004027


In [54]:
start = time.time()
res = vector_qa.invoke(HP.question11)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question11, res['result'], times)
print(res['result'])
print(times)

I can help you with that! According to the Harry Potter series, the entrance to the Chamber of Secrets is behind a tapestry in Moaning Myrtle's bathroom at Hogwarts School of Witchcraft and Wizardry. Thanks for asking!
1.6099642992019654


In [55]:
start = time.time()
res = vector_qa.invoke(HP.question12)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question12, res['result'], times)
print(res['result'])
print(times)

Based on the provided context, I don't have enough information to answer your question. The pieces of context only provide information about Draco Malfoy's name and positions held, but not specifically about a house elf serving the Malfoy family.

Thanks for asking!
1.7898398478825888


In [56]:
start = time.time()
res = vector_qa.invoke(HP.question13)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question13, res['result'], times)
print(res['result'])
print(times)

The name of the goblin that helps Harry break into Gringotts is Griphook. Thanks for asking!
1.4505933364232382


In [57]:
start = time.time()
res = vector_qa.invoke(HP.question14)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question14, res['result'], times)
print(res['result'])
print(times)

Sirius Black was James Sirius Potter's godfather and uncle, being the brother of Lily Evans Potter. Thanks for asking!
1.4409717957178751


In [58]:
start = time.time()
res = vector_qa.invoke(HP.question15)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question15, res['result'], times)
print(res['result'])
print(times)

I don't know the answer to this question. Thanks for asking!
1.2110074877738952


In [59]:
start = time.time()
res = vector_qa.invoke(HP.question16)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question16, res['result'], times)
print(res['result'])
print(times)

The context doesn't provide enough information to answer the question. The names mentioned are Helga Hufflepuff, Riddle-Harry (which seems incorrect), and Draco Malfoy, but none of them seem related to hippogriffs.

I don't know. Thanks for asking!
2.0092179695765178


In [60]:
start = time.time()
res = vector_qa.invoke(HP.question17)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question17, res['result'], times)
print(res['result'])
print(times)

Based on the provided context, I can tell you that there isn't any information about a specific ghost associated with Ravenclaw House. However, Hogwarts does have a ghostly student known as the White Lady, who is said to haunt the corridors of Ravenclaw Tower.

Thanks for asking!
1.8669166604677836


In [61]:
start = time.time()
res = vector_qa.invoke(HP.question18)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question18, res['result'], times)
print(res['result'])
print(times)

I don't know! Thanks for asking!
1.0642221609751383


In [62]:
start = time.time()
res = vector_qa.invoke(HP.question19)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question19, res['result'], times)
print(res['result'])
print(times)

Based on the provided context, I found information about three Weasleys: Ginevra, Barny, and Ron. Since their family name is "Weasley", it can be inferred that they are siblings.

Ginevra has 3 children, suggesting she is a parent, but her sibling count is not specified. Barny's and Ron's sibling counts are also unknown. However, we can assume all three are siblings since they share the same family name.

To answer your question: I don't know the exact number of Weasley siblings without more information. Thanks for asking!
2.6581976691881817


In [63]:
start = time.time()
res = vector_qa.invoke(HP.question20)
end = time.time()
times =(end - start)/60
log.LogChat(3, "graph", HP.question20, res['result'], times)
print(res['result'])
print(times)

According to the provided context, there are only three Weasley siblings mentioned: Ginevra, Panju, and Barny. Since you only provide information about their numbers of children, it seems that two out of these three siblings have more than one child (Ginevra has 3 children), while the third sibling's number of children is unknown.

However, without further information about the dates of birth or ages of each sibling, I cannot determine who the youngest Weasley sibling is. Therefore, my answer would be: "I don't know." Thanks for asking!
2.283704725901286


### Test af 20 forskellige spørgsmål til chatbot med sentence simularity

In [30]:
question = HP.question1
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Muggle Tom Riddle Snr
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0024373412132263185


In [31]:
question = HP.question2
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: the Weasleys
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.002593998114267985


In [32]:
question = HP.question3
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Bellatrix Lestrange
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.002361003557840983


In [33]:
question = HP.question4
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Merope Gaunt
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0026172677675882976


In [34]:
question = HP.question5
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Scorpius Malfoy
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.002589774131774902


In [35]:
question = HP.question6
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Lord Voldemort
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.002759059270222982


In [36]:
question = HP.question7
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Salazar Slytherin
positionHeld: founders
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0029218395551045737


In [37]:
question = HP.question8
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Albus Dumbledore
positionHeld: Headmaster
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.002525476614634196


In [38]:
question = HP.question9
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Lord Voldemort
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0028497099876403807


In [39]:
question = HP.question10
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Ravenclaw
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.002734243869781494


In [40]:
question = HP.question11
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Salazar Slytherin
positionHeld: founders
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0020970265070597333


In [41]:
question = HP.question12
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: WizardrySlytherinDraco Malfoy
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0027019818623860675


In [42]:
question = HP.question13
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Filius Flitwick
positionHeld: half-goblin
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0032722989718119303


In [43]:
question = HP.question14
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: James Sirius Potter
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0022852341334025066


In [44]:
question = HP.question15
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Helga Hufflepuff
positionHeld: founder
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.003341225783030192


In [45]:
question = HP.question16
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Riddle-Harry
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0030854463577270506


In [46]:
question = HP.question17
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Ravenclaw
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0027993162473042804


In [47]:
question = HP.question18
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: Ron Weasley
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0030919512112935386


In [48]:
question = HP.question19
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: the Weasleys
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.002094674110412598


In [49]:
question = HP.question20
start = time.time()
res = vector_index.similarity_search(question)
end = time.time()
times =(end - start)/60
log.LogChat(6, "graph_with_sentence_simularity", question, res[0].page_content, times)
print(res[0].page_content)
print(times)


name: the Weasleys
positionHeld: 
causeOfDeath: 
dateOfBirth: 
numberOfChildren: 
academicDegree: 
dateOfDeath: 
age: 
productType: 
foundingDate: 
0.0015493551890055338
